In [1]:
auth = '757F054AA22E1711B598A89EB276EDC193933EC8'

list_appid = ['tfy6d12dc-19a5-4093-9f00-f276d950f77',
              'picheng11-5163-4258-be21-a5629527a07',
              'Shanghai-b2aa-4eed-82c3-b1e42e2e9179',
              'danjidin-f8b1-4e84-adce-026944025ceb',
              'ibay365a-b93e-46e7-8b46-6a33b716ab2a',
              '-IBAY365-PRD-08e35c535-85b731b3']

In [2]:
import pandas as pd

In [3]:
df_menu = pd.read_excel('./menu.xlsx', header=0, dtype={'Item_Number': str})
list_item = df_menu['Item_Number'].to_list()

print('总数量：' + str(len(list_item)))
list_item

FileNotFoundError: [Errno 2] No such file or directory: './menu.xlsx'

In [ ]:
df = pd.DataFrame(columns=['Item_Number',
                           'Title',
                           'Price',
                           'Sold',
                           'Vehicle',
                           'Src',
                           'status'])

df

In [ ]:
import random
import requests
import json

In [ ]:
i = 0

index_appid = random.randint(0, 5)

developer = 'http://106.75.138.2/index.php/api/GetAppToken?auth=' + auth + '&appid=' + list_appid[index_appid]
resp = requests.get(developer).text
token = json.loads(resp)['getapptokenResponse']['token']

token

In [ ]:
headers = {'X-EBAY-API-IAF-TOKEN': token}
resp = requests.get('https://open.api.ebay.com/shopping?callname=GetSingleItem&appid=' + list_appid[index_appid] + '&responseencoding=JSON&siteid=0&version=967&IncludeSelector=Details,Compatibility,ItemSpecifics&ItemID=' + list_item[i], headers=headers).text

info = json.loads(resp)
if info['Ack'] != 'Success':
    print('Error')
else:
    print('Success')
    
info

In [ ]:
# Item.Title - None
title = info['Item']['Title']

title

In [ ]:
# Item.CurrentPrice - None
price = info['Item']['CurrentPrice']['CurrencyID'] + ' ' + str(info['Item']['CurrentPrice']['Value'])

price

In [ ]:
# Item.QuantitySold - Details
sold = info['Item']['QuantitySold']

sold

In [ ]:
# Item.ItemCompatibilityList - Compatibility
if 'ItemCompatibilityList' in info['Item']:
    vehicle = info['Item']['ItemCompatibilityList']['Compatibility']
else:
    vehicle = ''
    
vehicle

In [ ]:
dict_vehicle = {}
for list_name_value in vehicle:
    make = ''
    model = ''
    year = ''
    for name_value in list_name_value['NameValueList']:
        if 'Name' in name_value and 'Make' in name_value['Name']:
            make = name_value['Value'][0]
        elif 'Name' in name_value and 'Model' in name_value['Name']:
            model = name_value['Value'][0]
        elif 'Name' in name_value and 'Year' in name_value['Name']:
            year = name_value['Value'][0]
            
    if make not in dict_vehicle:
        dict_vehicle[make] = {model: [year]}
    else:
        if model not in dict_vehicle[make]:
            dict_vehicle[make][model] = [year]
        else:
            dict_vehicle[make][model].append(year)
            
list_vehicle = []
for make in dict_vehicle:
    for model in dict_vehicle[make]:
        list_vehicle.append(make + ' ' + model + ' ' + min(dict_vehicle[make][model]) + '-' + max(dict_vehicle[make][model]))
        
vehicle = '\n'.join(list_vehicle)

print(vehicle)

In [ ]:
# dict_vehicle = {}
# for name_value in vehicle:
#     make = name_value['NameValueList'][2]['Value'][0]
#     model = name_value['NameValueList'][3]['Value'][0]
#     try:
#         engine = re.findall(r'^[.0-9]+L', name_value['NameValueList'][5]['Value'][0])[0]
#     except:
#         continue
#     year = name_value['NameValueList'][1]['Value'][0]

#     if make not in dict_vehicle:
#         dict_vehicle[make] = {model: {engine: [year]}}
#     else:
#         if model not in dict_vehicle[make]:
#             dict_vehicle[make][model] = {engine: [year]}
#         else:
#             if engine not in dict_vehicle[make][model]:
#                 dict_vehicle[make][model][engine] = [year]
#             else:
#                 dict_vehicle[make][model][engine].append(year)

# list_vehicle = []
# for make in dict_vehicle:
#     for model in dict_vehicle[make]:
#         for engine in dict_vehicle[make][model]:
#             list_vehicle.append(make + ' ' + model + ' ' + engine + ' ' + min(dict_vehicle[make][model][engine]) + '-' + max(dict_vehicle[make][model][engine]))

# vehicle = '\n'.join(list_vehicle)

# print(vehicle)

In [ ]:
# Item.GalleryURL - None
if 'GalleryURL' in info['Item']:
    src = info['Item']['GalleryURL']
elif 'PictureURL' in info['Item']:
    if len(info['Item']['PictureURL']) == 0:
        src = ''
    else:
        src = info['Item']['PictureURL'][0]
        
src

In [ ]:
# Item.ItemSpecifics - ItemSpecifics
if 'ItemSpecifics' in info['Item']:
    details = info['Item']['ItemSpecifics']['NameValueList']
else:
    details = ''
    
details

In [ ]:
df_temp = pd.DataFrame([{'Item_Number': list_item[i],
                         'Title': title,
                         'Price': price,
                         'Sold': sold,
                         'Vehicle': vehicle,
                         'Src': src,
                         'status': 'ok'}])

df_temp

In [ ]:
for name_value in details:
    df_temp[name_value['Name'].lower()] = ';'.join(name_value['Value'])
    
df = pd.concat([df, df_temp], ignore_index=True).fillna('')

df